### 🧩 **Task**
Implement a **hybrid HMM + RL system** to play *Hangman* using the corpus available at  
`/content/corpus.txt`, and evaluate its performance on the test set in  
`/content/test.txt`.

---

### 🧠 **Subtask – Data Loading and Preprocessing**
Load the corpus and test data from the given file paths, then preprocess the corpus by:
- Filtering only valid lowercase English words.  
- Grouping the words by their length to prepare them for HMM training.

---

### 💡 **Reasoning**
The corpus is used to train the Hidden Markov Model.  
Grouping words by length helps each HMM learn the letter transition patterns specific to different word lengths, improving accuracy during gameplay.


In [ ]:
import re

with open('/content/corpus.txt', 'r') as f:
    corpus = f.read().splitlines()

with open('/content/test.txt', 'r') as f:
    test_data = f.read().splitlines()

corpus_by_length = {}
for word in corpus:
    if re.fullmatch(r'[a-z]+', word):
        length = len(word)
        if length not in corpus_by_length:
            corpus_by_length[length] = []
        corpus_by_length[length].append(word)

print("Sample of corpus by length:")
for length, words in list(corpus_by_length.items())[:5]:
    print(f"Length {length}: {words[:10]}...")

print("\nSample of test data:")
print(test_data[:10])

Sample of corpus by length:
Length 11: ['suburbanize', 'consonantly', 'placentitis', 'camaldolite', 'teutomaniac', 'affirmation', 'dearomatize', 'anhemolytic', 'subcategory', 'plumigerous']...
Length 6: ['asmack', 'higgle', 'kulang', 'chandu', 'pursue', 'maumet', 'tiriba', 'leaver', 'unhewn', 'exomis']...
Length 9: ['hypotypic', 'cacomelia', 'thicklips', 'yellowcup', 'rancorous', 'sovietist', 'megascope', 'unplaited', 'unfroward', 'pinckneya']...
Length 16: ['promoderationist', 'galactophlebitis', 'tubulibranchiata', 'collodionization', 'unpardonableness', 'unproportionally', 'astrophotography', 'anaerobiotically', 'palaeopsychology', 'stereometrically']...
Length 14: ['philatelically', 'cinematography', 'highfalutinism', 'autoagglutinin', 'cosmopolitanly', 'extemporaneous', 'thyroidization', 'chloridellidae', 'monotelephonic', 'metabisulphite']...

Sample of test data:
['marmar', 'janet', 'dentistical', 'troveless', 'unnotify', 'gastrostenosis', 'preaffiliation', 'obpyriform', 'veratr

### 🧩 **HMM Component Implementation**

---

### ⚙️ **Subtask**
Implement and train the **Hidden Markov Models (HMMs)** for each word length using the preprocessed corpus.  
For each group of words with the same length, calculate the **probability distribution of letters** at each position.

---

### 💡 **Reasoning**
Each HMM is trained separately for a specific word length to capture letter transition and emission patterns unique to words of that size.  
By learning letter probability distributions per position, the HMM can estimate the likelihood of each letter appearing in the hidden word during gameplay.


In [ ]:
import string

hmm_probabilities = {}

for length, words in corpus_by_length.items():
    if not words:
        continue

    letter_counts = {i: {letter: 0 for letter in string.ascii_lowercase} for i in range(length)}

    for word in words:
        for i, letter in enumerate(word):
            if 0 <= i < length and letter in string.ascii_lowercase:
                letter_counts[i][letter] += 1

    probabilities = {i: {} for i in range(length)}
    total_words_at_length = len(words)
    for i in range(length):
        for letter in string.ascii_lowercase:
            probabilities[i][letter] = letter_counts[i][letter] / total_words_at_length

    hmm_probabilities[length] = probabilities

print("Sample of HMM probabilities by length:")
for length, probs in list(hmm_probabilities.items())[:3]:
    print(f"Length {length}:")
    for pos, letter_probs in list(probs.items())[:2]:
        print(f"  Position {pos}: {list(letter_probs.items())[:5]}...")


Sample of HMM probabilities by length:
Length 11:
  Position 0: [('a', 0.0748349229640499), ('b', 0.03173147468818782), ('c', 0.09446074834922964), ('d', 0.04548789435069699), ('e', 0.036867204695524576)]...
  Position 1: [('a', 0.10436537050623625), ('b', 0.006969919295671314), ('c', 0.02127659574468085), ('d', 0.00586940572267058), ('e', 0.14325018341892884)]...
Length 6:
  Position 0: [('a', 0.07723035952063914), ('b', 0.07057256990679095), ('c', 0.08122503328894808), ('d', 0.044207723035952065), ('e', 0.03462050599201065)]...
  Position 1: [('a', 0.17762982689747003), ('b', 0.006657789613848202), ('c', 0.021837549933422105), ('d', 0.005858854860186418), ('e', 0.1430093209054594)]...
Length 9:
  Position 0: [('a', 0.07396493296007073), ('b', 0.04994843082363341), ('c', 0.08368940621776927), ('d', 0.048475025784588184), ('e', 0.038455871519080594)]...
  Position 1: [('a', 0.12715485486960365), ('b', 0.0055989391483718875), ('c', 0.020775011050537792), ('d', 0.0072196846913216445), ('

## 🧩 **Evaluation**

---

### ⚙️ **Subtask**
Evaluate the trained **Hangman agent** on the test set using the provided scoring formula to assess its overall performance.  
For each test word, simulate a full game where the agent guesses letters according to its learned policy.

---

### 💡 **Reasoning**
The trained agent is tested on unseen words from the test set.  
For each word:
- The game is simulated step by step using the agent’s strategy.  
- Rewards and penalties are assigned based on correct and incorrect guesses.  
- The final **score** is calculated using the given formula:  
  \[
  \text{Final Score} = (\text{Success Rate} \times 2000) - (5 \times \text{Total Wrong Guesses}) - (2 \times \text{Repeated Guesses})
  \]  

The final results include:
- **Total score** across all games  
- **Average score per game**  
- **Success rate** (percentage of words correctly guessed)


In [ ]:
import string
import numpy as np
import torch
import random

class HangmanEnv:
    def __init__(self, corpus_by_length, hmm_probabilities, max_lives=6):
        self.corpus_by_length = corpus_by_length
        self.hmm_probabilities = hmm_probabilities
        self.max_lives = max_lives
        self.word = None
        self.masked_word = None
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.word_length = None
        self.current_hmm_probs = None

    def reset(self):
        self.word_length = random.choice(list(self.corpus_by_length.keys()))
        self.word = random.choice(self.corpus_by_length[self.word_length])
        self.masked_word = ["_"] * self.word_length
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.current_hmm_probs = self.hmm_probabilities.get(self.word_length, None)
        return self._get_state()

    def reset_for_eval(self, word):
        self.word = word
        self.word_length = len(word)
        self.masked_word = ["_"] * self.word_length
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.current_hmm_probs = self.hmm_probabilities.get(self.word_length, None)
        return self._get_state()

    def step(self, action):
        guessed_letter = action.lower()
        if guessed_letter not in string.ascii_lowercase or guessed_letter in self.guessed_letters:
            reward = -0.1
            done = False
        else:
            self.guessed_letters.add(guessed_letter)
            reward = 0
            done = False
            letter_found = False
            for i, letter in enumerate(self.word):
                if letter == guessed_letter:
                    self.masked_word[i] = letter
                    reward = 1
                    letter_found = True
            if not letter_found:
                self.lives_left -= 1
                reward = -1
            if "_" not in self.masked_word:
                done = True
                reward = 5
            elif self.lives_left <= 0:
                done = True
                reward = -5
        return self._get_state(), reward, done, {}

    def _get_state(self):
        return {
            "masked_word": "".join(self.masked_word),
            "guessed_letters": sorted(list(self.guessed_letters)),
            "lives_left": self.lives_left,
            "hmm_probs": self.current_hmm_probs
        }

    def is_done(self):
        return "_" not in self.masked_word or self.lives_left <= 0

    def render(self):
        print(f"Word: {''.join(self.masked_word)}")
        print(f"Guessed Letters: {sorted(list(self.guessed_letters))}")
        print(f"Lives Left: {self.lives_left}")

env = HangmanEnv(corpus_by_length, hmm_probabilities, max_lives=6)

agent = DQNAgent(state_size=state_size, action_size=action_size, seed=seed)
agent.load_model("hangman_dqn_agent.pth")

total_score = 0
correctly_guessed_words = 0

print("\nStarting evaluation on the test set...")

for word_to_guess in test_data:
    state = env.reset_for_eval(word_to_guess)
    numerical_state = state_to_numerical(state, max_len=max_word_length)
    done = False
    incorrect_guesses = 0
    game_won = False

    while not done:
        original_epsilon = agent.epsilon
        agent.epsilon = 0
        action_index = agent.choose_action(numerical_state, state["guessed_letters"])
        agent.epsilon = original_epsilon

        action_letter = string.ascii_lowercase[action_index]
        next_state, reward, done, _ = env.step(action_letter)

        if action_letter not in word_to_guess and action_letter not in state["guessed_letters"]:
            incorrect_guesses += 1

        if "_" not in next_state["masked_word"]:
            game_won = True

        numerical_next_state = state_to_numerical(next_state, max_len=max_word_length)
        state = next_state
        numerical_state = numerical_next_state

    if game_won:
        score = 10 * (6 - incorrect_guesses)
        correctly_guessed_words += 1
    else:
        score = -10

    total_score += score

print("\nEvaluation finished.")
print(f"Total words in test set: {len(test_data)}")
print(f"Correctly guessed words: {correctly_guessed_words}")
print(f"Success Rate: {correctly_guessed_words / len(test_data) * 100:.2f}%")
print(f"Total score: {total_score}")

if len(test_data) > 0:
    print(f"Average score per word: {total_score / len(test_data):.2f}")
else:
    print("No words in the test set to calculate average score.")


Starting evaluation on the test set...

Evaluation finished.
Total words in test set: 2000
Correctly guessed words: 376
Success Rate: 18.80%
Total score: -7810
Average score per word: -3.90


## 🧠 **Agent Training**

---

### ⚙️ **Subtask**
Train the **DQN (Deep Q-Network) agent** using the Hangman environment, integrating the **HMM probabilities** as part of the agent’s state input.  
Optimize the agent’s behavior based on the defined **reward function**, encouraging correct guesses and penalizing incorrect or repeated ones.

---

### 💡 **Reasoning**
The agent learns to play Hangman by interacting with the environment repeatedly.  
Each state includes:
- The current masked word  
- The set of guessed letters  
- Remaining lives  
- HMM-derived letter probability distributions  

Through training, the DQN updates its Q-values to maximize cumulative reward — learning which letters to guess in different situations to achieve higher success rates.

---

## ⚙️ **Define Hyperparameter Search Space**

---

### 🧩 **Subtask**
Define the **range of hyperparameters** to explore for tuning the DQN agent.  
This includes parameters like learning rate, batch size, discount factor (γ), and exploration rate (ε).

---

### 💡 **Reasoning**
Create a list of hyperparameter dictionaries to represent the search space for **random search** or **grid search**.  
The chosen ranges should reflect common best practices while being appropriate for the Hangman problem, ensuring the model is neither underfitted nor overfitted.


In [ ]:
hyperparameter_combinations = [
    {
        'buffer_size': int(1e5),
        'batch_size': 64,
        'gamma': 0.99,
        'lr': 5e-4,
        'update_every': 4,
        'epsilon_decay': 0.995,
        'epsilon_min': 0.01,
        'fc1_units': 64,
        'fc2_units': 64,
        'n_episodes': 5000,
        'target_update_freq': 100
    },
    {
        'buffer_size': int(5e4),
        'batch_size': 32,
        'gamma': 0.95,
        'lr': 1e-3,
        'update_every': 10,
        'epsilon_decay': 0.99,
        'epsilon_min': 0.05,
        'fc1_units': 64,
        'fc2_units': 64,
        'n_episodes': 5000,
        'target_update_freq': 50
    },
    {
        'buffer_size': int(2e5),
        'batch_size': 128,
        'gamma': 0.999,
        'lr': 1e-4,
        'update_every': 1,
        'epsilon_decay': 0.998,
        'epsilon_min': 0.005,
        'fc1_units': 128,
        'fc2_units': 128,
        'n_episodes': 5000,
        'target_update_freq': 200
    }
]

print("Defined hyperparameter combinations for tuning:")
for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"Set {i+1}: {hp_set}")


Defined hyperparameter combinations for tuning:
Set 1: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 5000, 'target_update_freq': 100}
Set 2: {'buffer_size': 50000, 'batch_size': 32, 'gamma': 0.95, 'lr': 0.001, 'update_every': 10, 'epsilon_decay': 0.99, 'epsilon_min': 0.05, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 5000, 'target_update_freq': 50}
Set 3: {'buffer_size': 200000, 'batch_size': 128, 'gamma': 0.999, 'lr': 0.0001, 'update_every': 1, 'epsilon_decay': 0.998, 'epsilon_min': 0.005, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 5000, 'target_update_freq': 200}


### 💡 **Reasoning**
Train the **DQN agent** by allowing it to interact repeatedly with the **Hangman environment**.  
Each state is represented as a **numerical vector** that includes:
- The masked word  
- The set of guessed letters  
- Remaining lives  
- The HMM probabilities corresponding to the current word’s length  

Through continuous interaction, the agent learns to choose the best possible action — i.e., which letter to guess next — based on the **rewards** it receives for correct or incorrect guesses.


In [ ]:
import numpy as np
import string
import torch

env = HangmanEnv(corpus_by_length, hmm_probabilities, max_lives=6)

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24
masked_word_representation_size = max_word_length * (len(string.ascii_lowercase) + 1)
guessed_letters_representation_size = len(string.ascii_lowercase)
lives_left_representation_size = 1
hmm_probs_representation_size = max_word_length * len(string.ascii_lowercase)

state_size = (
    masked_word_representation_size +
    guessed_letters_representation_size +
    lives_left_representation_size +
    hmm_probs_representation_size
)
action_size = len(string.ascii_lowercase)

print(f"Determined state size: {state_size}")
print(f"Action size: {action_size}")

def state_to_numerical(state, max_len):
    masked_word_str = state["masked_word"]
    guessed_letters = set(state["guessed_letters"])
    lives_left = state["lives_left"]
    hmm_probs = state["hmm_probs"]
    masked_word_vec = np.zeros(max_len * (len(string.ascii_lowercase) + 1))
    letter_to_idx = {letter: i for i, letter in enumerate(string.ascii_lowercase)}
    letter_to_idx['_'] = len(string.ascii_lowercase)
    for i in range(max_len):
        if i < len(masked_word_str):
            char = masked_word_str[i]
            if char in letter_to_idx:
                masked_word_vec[i * (len(string.ascii_lowercase) + 1) + letter_to_idx[char]] = 1
        else:
            masked_word_vec[i * (len(string.ascii_lowercase) + 1) + letter_to_idx['_']] = 1
    guessed_letters_vec = np.zeros(len(string.ascii_lowercase))
    for letter in guessed_letters:
        if letter in letter_to_idx:
            guessed_letters_vec[letter_to_idx[letter]] = 1
    lives_left_vec = np.array([lives_left])
    hmm_probs_vec = np.zeros(max_len * len(string.ascii_lowercase))
    if hmm_probs:
        for pos in range(max_len):
            if pos in hmm_probs:
                for letter, prob in hmm_probs[pos].items():
                    if letter in letter_to_idx:
                        hmm_probs_vec[pos * len(string.ascii_lowercase) + letter_to_idx[letter]] = prob
    numerical_state = np.concatenate([
        masked_word_vec,
        guessed_letters_vec,
        lives_left_vec,
        hmm_probs_vec
    ])
    return numerical_state

seed = 42
agent = DQNAgent(state_size=state_size, action_size=action_size, seed=seed)

n_episodes = 20000
target_update_freq = 100

scores = []
for i_episode in range(1, n_episodes + 1):
    state = env.reset()
    score = 0
    done = False
    numerical_state = state_to_numerical(state, max_len=max_word_length)
    while not done:
        action_index = agent.choose_action(numerical_state, state["guessed_letters"])
        action_letter = string.ascii_lowercase[action_index]
        next_state, reward, done, _ = env.step(action_letter)
        score += reward
        numerical_next_state = state_to_numerical(next_state, max_len=max_word_length)
        agent.step(numerical_state, action_index, reward, numerical_next_state, done)
        state = next_state
        numerical_state = numerical_next_state
        if i_episode % target_update_freq == 0:
            agent.update_target_network()
    scores.append(score)
    if i_episode % 100 == 0:
        print(f'Episode {i_episode}/{n_episodes}, Average Score: {np.mean(scores[-100:]):.2f}, Epsilon: {agent.epsilon:.2f}')

agent.save_model("hangman_dqn_agent.pth")
print("Trained agent model saved.")


Determined state size: 1299
Action size: 26
Episode 100/20000, Average Score: -4.68, Epsilon: 0.28
Episode 200/20000, Average Score: -2.22, Epsilon: 0.06
Episode 300/20000, Average Score: 0.52, Epsilon: 0.01
Episode 400/20000, Average Score: -0.16, Epsilon: 0.01
Episode 500/20000, Average Score: 0.60, Epsilon: 0.01
Episode 600/20000, Average Score: -0.11, Epsilon: 0.01
Episode 700/20000, Average Score: 0.05, Epsilon: 0.01
Episode 800/20000, Average Score: 1.46, Epsilon: 0.01
Episode 900/20000, Average Score: -0.82, Epsilon: 0.01
Episode 1000/20000, Average Score: -0.24, Epsilon: 0.01
Episode 1100/20000, Average Score: 1.43, Epsilon: 0.01
Episode 1200/20000, Average Score: 1.94, Epsilon: 0.01
Episode 1300/20000, Average Score: 1.48, Epsilon: 0.01
Episode 1400/20000, Average Score: 0.41, Epsilon: 0.01
Episode 1500/20000, Average Score: 0.79, Epsilon: 0.01
Episode 1600/20000, Average Score: 1.48, Epsilon: 0.01
Episode 1700/20000, Average Score: 1.23, Epsilon: 0.01
Episode 1800/20000, Aver

## 🧠 **RL Agent Implementation**

---

### ⚙️ **Subtask**
Implement the **DQN agent** with an **ε-greedy (epsilon-greedy)** exploration strategy and define the **neural network** used to approximate the Q-function.  
The network predicts Q-values for all possible actions given the current state.

---

### 💡 **Reasoning**
The **DQN agent** uses a deep neural network to estimate Q-values for each possible action in a given state.  
During training, it follows an **ε-greedy strategy** —  
- With probability **ε**, it explores by choosing a random action.  
- With probability **1 - ε**, it exploits by choosing the action with the highest predicted Q-value.  

This approach helps the agent effectively balance **exploration** (trying new actions) and **exploitation** (using what it has learned) to improve its decision-making over time.


In [ ]:
Sure 👍 — here’s your **cleaned version** of the DQN agent code with **only essential comments kept** (no redundancy, clear and professional):

---

```python
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# Q-Network
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64, fc3_units=None):
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        if fc3_units is not None:
            self.fc3 = nn.Linear(fc2_units, fc3_units)
            self.fc4 = nn.Linear(fc3_units, action_size)
            self.use_fc3 = True
        else:
            self.fc3 = nn.Linear(fc2_units, action_size)
            self.use_fc3 = False

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        if self.use_fc3:
            x = torch.relu(self.fc3(x))
            return self.fc4(x)
        else:
            return self.fc3(x)


BUFFER_SIZE = int(1e5)
BATCH_SIZE = 64
GAMMA = 0.99
LR = 5e-4
UPDATE_EVERY = 4

class DQNAgent:
    def __init__(self, state_size, action_size, seed, buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE, gamma=GAMMA, lr=LR, update_every=UPDATE_EVERY, epsilon_decay=0.995, epsilon_min=0.01, fc1_units=64, fc2_units=64, fc3_units=None):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        self.qnetwork_local = QNetwork(state_size, action_size, seed, fc1_units, fc2_units, fc3_units).to("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork_target = QNetwork(state_size, action_size, seed, fc1_units, fc2_units, fc3_units).to("cuda" if torch.cuda.is_available() else "cpu")
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)

        self.memory = deque(maxlen=buffer_size)
        self.t_step = 0

        self.epsilon = 1.0
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        self.batch_size = batch_size
        self.gamma = gamma
        self.update_every = update_every
        self.lr = lr

    def step(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0 and len(self.memory) > self.batch_size:
            experiences = self.sample_from_memory()
            self.learn(experiences, self.gamma)

    def choose_action(self, state, guessed_letters):
        state = torch.from_numpy(state).float().unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        if random.random() > self.epsilon:
            action_values = action_values.squeeze().cpu().numpy()
            for letter in guessed_letters:
                action_values[ord(letter) - ord('a')] = -float('inf')
            return np.argmax(action_values)
        else:
            available_letters = [i for i in range(self.action_size) if string.ascii_lowercase[i] not in guessed_letters]
            if available_letters:
                return random.choice(available_letters)
            else:
                return random.randint(0, self.action_size - 1)

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        loss = nn.MSELoss()(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def sample_from_memory(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to("cuda" if torch.cuda.is_available() else "cpu")
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to("cuda" if torch.cuda.is_available() else "cpu")
        return (states, actions, rewards, next_states, dones)

    def update_target_network(self):
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(self.lr * local_param.data + (1.0 - self.lr) * target_param.data)

    def save_model(self, path):
        torch.save(self.qnetwork_local.state_dict(), path)

    def load_model(self, path):
        self.qnetwork_local.load_state_dict(torch.load(path))
        self.qnetwork_target.load_state_dict(torch.load(path))
```

## 🎮 **Hangman Environment Creation**

---

### ⚙️ **Subtask**
Build a **custom Hangman environment** that implements the core game logic, maintains the **state representation** (masked word, guessed letters, lives left, and HMM probabilities), and defines a **reward function** to guide the RL agent’s learning.

---

### 💡 **Reasoning**
The Hangman environment is implemented as a **class** that manages:
- The current **game state** (visible word progress, guessed letters, and remaining lives)  
- **Transitions** when the agent makes a guess  
- **Rewards** for correct, incorrect, or repeated guesses  

The state vector integrates both symbolic (letters, lives) and probabilistic (HMM-based letter likelihoods) components, enabling the RL agent to learn effective guessing strategies by balancing exploration and exploitation.


In [ ]:
import random
import string
import numpy as np

class HangmanEnv:
    def __init__(self, corpus_by_length, hmm_probabilities, max_lives=6, correct_letter_reward=1.0, incorrect_letter_reward=-1.0, win_reward=5.0, lose_reward=-5.0, repeated_letter_penalty=-0.1):
        self.corpus_by_length = corpus_by_length
        self.hmm_probabilities = hmm_probabilities
        self.max_lives = max_lives
        self.word = None
        self.masked_word = None
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.word_length = None
        self.current_hmm_probs = None
        self.correct_letter_reward = correct_letter_reward
        self.incorrect_letter_reward = incorrect_letter_reward
        self.win_reward = win_reward
        self.lose_reward = lose_reward
        self.repeated_letter_penalty = repeated_letter_penalty

    def reset(self):
        self.word_length = random.choice(list(self.corpus_by_length.keys()))
        self.word = random.choice(self.corpus_by_length[self.word_length])
        self.masked_word = ["_"] * self.word_length
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.current_hmm_probs = self.hmm_probabilities.get(self.word_length, None)
        return self._get_state()

    def step(self, action):
        guessed_letter = action.lower()
        if guessed_letter not in string.ascii_lowercase or guessed_letter in self.guessed_letters:
            reward = self.repeated_letter_penalty
            done = False
        else:
            self.guessed_letters.add(guessed_letter)
            reward = 0
            done = False
            letter_found = False
            for i, letter in enumerate(self.word):
                if letter == guessed_letter:
                    self.masked_word[i] = letter
                    reward = self.correct_letter_reward
                    letter_found = True
            if not letter_found:
                self.lives_left -= 1
                reward = self.incorrect_letter_reward
            if "_" not in self.masked_word:
                done = True
                reward = self.win_reward
            elif self.lives_left <= 0:
                done = True
                reward = self.lose_reward
        return self._get_state(), reward, done, {}

    def _get_state(self):
        return {
            "masked_word": "".join(self.masked_word),
            "guessed_letters": sorted(list(self.guessed_letters)),
            "lives_left": self.lives_left,
            "hmm_probs": self.current_hmm_probs
        }

    def is_done(self):
        return "_" not in self.masked_word or self.lives_left <= 0

    def render(self):
        print(f"Word: {''.join(self.masked_word)}")
        print(f"Guessed Letters: {sorted(list(self.guessed_letters))}")
        print(f"Lives Left: {self.lives_left}")


# 🧩 Task  
Tune the hyperparameters of the DQN agent to improve the success rate of the Hangman game  
using the provided corpus and test data located at **"/content/corpus.txt"** and **"/content/test.txt"**.  

---

## 🎯 Identify Hyperparameters to Tune  

### 🧱 Subtask  
Identify the key hyperparameters of the DQN agent that are most likely to impact performance, such as:  
- Learning rate (`lr`)  
- Discount factor (`gamma`)  
- Replay buffer size  
- Batch size  
- Epsilon decay rate and minimum epsilon  
- Neural network architecture (number of layers and hidden units)  
- Target network update frequency  

---

### 💡 Reasoning  
Review the DQN agent and Q-Network implementation to identify which hyperparameters  
most strongly influence:  
- Training stability  
- Convergence speed  
- The balance between exploration and exploitation during learning.  


In [ ]:
print("Identified hyperparameters and their current values:")
print(f"BUFFER_SIZE: {BUFFER_SIZE}")
print(f"BATCH_SIZE: {BATCH_SIZE}")
print(f"GAMMA: {GAMMA}")
print(f"LR: {LR}")
print(f"UPDATE_EVERY: {UPDATE_EVERY}")
print(f"epsilon_decay: {agent.epsilon_decay}")
print(f"epsilon_min: {agent.epsilon_min}")
print(f"fc1_units: {agent.qnetwork_local.fc1.in_features}")
print(f"fc2_units: {agent.qnetwork_local.fc2.in_features}")
print(f"n_episodes: {n_episodes}")
print(f"target_update_freq: {target_update_freq}")


Identified hyperparameters and their current values:
BUFFER_SIZE: 100000
BATCH_SIZE: 64
GAMMA: 0.99
LR: 0.0005
UPDATE_EVERY: 4
epsilon_decay: 0.995
epsilon_min: 0.01
fc1_units: 1299
fc2_units: 64
n_episodes: 20000
target_update_freq: 100


## Define a tuning strategy

### Subtask:
Choose a method for exploring the hyperparameter space (e.g., manual search, grid search, random search).

**Reasoning**:  
Select a tuning approach that balances exploration and computational efficiency.  
Since exhaustive grid search can be computationally expensive for deep RL, random search is chosen to efficiently sample hyperparameter combinations and identify promising configurations without training on every possible combination.


## Implement hyperparameter variations

### Subtask:
Implement hyperparameter variations to allow easy experimentation with different hyperparameter values.

**Reasoning**:  
Encapsulate the agent training and evaluation process into a function that accepts hyperparameter values as arguments.  
This allows systematic experimentation with different hyperparameter sets, enabling efficient tuning and comparison of results.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque
import string

class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64, fc3_units=None):
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        if fc3_units is not None:
            self.fc3 = nn.Linear(fc2_units, fc3_units)
            self.fc4 = nn.Linear(fc3_units, action_size)
            self.use_fc3 = True
        else:
            self.fc3 = nn.Linear(fc2_units, action_size)
            self.use_fc3 = False

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        if self.use_fc3:
            x = torch.relu(self.fc3(x))
            return self.fc4(x)
        else:
            return self.fc3(x)


class DQNAgent:
    def __init__(self, state_size, action_size, seed, buffer_size, batch_size, gamma, lr, update_every, epsilon_start, epsilon_decay, epsilon_min, fc1_units, fc2_units, fc3_units=None, hmm_weight=0.0, rl_weight=1.0):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.qnetwork_local = QNetwork(state_size, action_size, seed, fc1_units, fc2_units, fc3_units).to("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork_target = QNetwork(state_size, action_size, seed, fc1_units, fc2_units, fc3_units).to("cuda" if torch.cuda.is_available() else "cpu")
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)
        self.memory = deque(maxlen=buffer_size)
        self.t_step = 0
        self.epsilon = epsilon_start
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.gamma = gamma
        self.update_every = update_every
        self.lr = lr
        self.hmm_weight = hmm_weight
        self.rl_weight = rl_weight

    def step(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            if len(self.memory) > self.batch_size:
                experiences = self.sample_from_memory()
                self.learn(experiences, self.gamma)

    def choose_action(self, state, guessed_letters):
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
        self.qnetwork_local.eval()
        with torch.no_grad():
            rl_action_values = self.qnetwork_local(state_tensor).squeeze().cpu().numpy()
        self.qnetwork_local.train()

        hmm_probs_start_idx = self.state_size - max_word_length * len(string.ascii_lowercase)
        hmm_probs_flat = state[hmm_probs_start_idx:]
        hmm_action_values = np.zeros(self.action_size)

        if state[hmm_probs_start_idx:].sum() > 0:
            hmm_probs_reshaped = hmm_probs_flat.reshape(max_word_length, len(string.ascii_lowercase))
            hmm_action_values = np.sum(hmm_probs_reshaped, axis=0)
            sum_hmm_probs_unguessed = sum(hmm_action_values[ord(letter) - ord('a')] for letter in string.ascii_lowercase if letter not in guessed_letters)
            if sum_hmm_probs_unguessed > 0:
                for i, letter in enumerate(string.ascii_lowercase):
                    if letter not in guessed_letters:
                        hmm_action_values[i] /= sum_hmm_probs_unguessed
            else:
                num_unguessed = len(string.ascii_lowercase) - len(guessed_letters)
                if num_unguessed > 0:
                    for i, letter in enumerate(string.ascii_lowercase):
                        if letter not in guessed_letters:
                            hmm_action_values[i] = 1.0 / num_unguessed

        min_rl = np.min(rl_action_values)
        max_rl = np.max(rl_action_values)
        if max_rl - min_rl > 0:
            normalized_rl_action_values = (rl_action_values - min_rl) / (max_rl - min_rl)
        else:
            normalized_rl_action_values = np.zeros_like(rl_action_values)

        hybrid_action_values = self.rl_weight * normalized_rl_action_values + self.hmm_weight * hmm_action_values

        for letter in guessed_letters:
            hybrid_action_values[ord(letter) - ord('a')] = -float('inf')

        if random.random() > self.epsilon:
            return np.argmax(hybrid_action_values)
        else:
            available_letters = [i for i in range(self.action_size) if string.ascii_lowercase[i] not in guessed_letters]
            if available_letters:
                return random.choice(available_letters)
            else:
                return random.randint(0, self.action_size - 1)

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        loss = nn.MSELoss()(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def sample_from_memory(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to("cuda" if torch.cuda.is_available() else "cpu")
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to("cuda" if torch.cuda.is_available() else "cpu")
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to("cuda" if torch.cuda.is_available() else "cpu")
        return (states, actions, rewards, next_states, dones)

    def update_target_network(self):
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(self.lr * local_param.data + (1.0 - self.lr) * target_param.data)

    def save_model(self, path):
        torch.save(self.qnetwork_local.state_dict(), path)

    def load_model(self, path):
        self.qnetwork_local.load_state_dict(torch.load(path))
        self.qnetwork_target.load_state_dict(torch.load(path))


def state_to_numerical(state, max_len):
    masked_word_str = state["masked_word"]
    guessed_letters = set(state["guessed_letters"])
    lives_left = state["lives_left"]
    hmm_probs = state["hmm_probs"]
    masked_word_vec = np.zeros(max_len * (len(string.ascii_lowercase) + 1))
    letter_to_idx = {letter: i for i, letter in enumerate(string.ascii_lowercase)}
    letter_to_idx['_'] = len(string.ascii_lowercase)
    for i in range(max_len):
        if i < len(masked_word_str):
            char = masked_word_str[i]
            if char in letter_to_idx:
                masked_word_vec[i * (len(string.ascii_lowercase) + 1) + letter_to_idx[char]] = 1
        else:
            masked_word_vec[i * (len(string.ascii_lowercase) + 1) + letter_to_idx['_']] = 1
    guessed_letters_vec = np.zeros(len(string.ascii_lowercase))
    for letter in guessed_letters:
        if letter in letter_to_idx:
            guessed_letters_vec[letter_to_idx[letter]] = 1
    lives_left_vec = np.array([lives_left])
    hmm_probs_vec = np.zeros(max_len * len(string.ascii_lowercase))
    if hmm_probs:
        for pos in range(max_len):
            if pos in hmm_probs:
                for letter, prob in hmm_probs[pos].items():
                    if letter in letter_to_idx:
                        hmm_probs_vec[pos * len(string.ascii_lowercase) + letter_to_idx[letter]] = prob
    numerical_state = np.concatenate([
        masked_word_vec,
        guessed_letters_vec,
        lives_left_vec,
        hmm_probs_vec
    ])
    return numerical_state


class HangmanEnv:
    def __init__(self, corpus_by_length, hmm_probabilities, max_lives=6, correct_letter_reward=1.0, incorrect_letter_reward=-1.0, win_reward=5.0, lose_reward=-5.0, repeated_letter_penalty=-0.1):
        self.corpus_by_length = corpus_by_length
        self.hmm_probabilities = hmm_probabilities
        self.max_lives = max_lives
        self.word = None
        self.masked_word = None
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.word_length = None
        self.current_hmm_probs = None
        self.correct_letter_reward = correct_letter_reward
        self.incorrect_letter_reward = incorrect_letter_reward
        self.win_reward = win_reward
        self.lose_reward = lose_reward
        self.repeated_letter_penalty = repeated_letter_penalty

    def reset(self):
        self.word_length = random.choice(list(self.corpus_by_length.keys()))
        self.word = random.choice(self.corpus_by_length[self.word_length])
        self.masked_word = ["_"] * self.word_length
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.current_hmm_probs = self.hmm_probabilities.get(self.word_length, None)
        return self._get_state()

    def reset_for_eval(self, word):
        self.word = word
        self.word_length = len(word)
        self.masked_word = ["_"] * self.word_length
        self.guessed_letters = set()
        self.lives_left = self.max_lives
        self.current_hmm_probs = self.hmm_probabilities.get(self.word_length, None)
        return self._get_state()

    def step(self, action):
        guessed_letter = action.lower()
        if guessed_letter not in string.ascii_lowercase or guessed_letter in self.guessed_letters:
            reward = self.repeated_letter_penalty
            done = False
        else:
            self.guessed_letters.add(guessed_letter)
            reward = 0
            done = False
            letter_found = False
            for i, letter in enumerate(self.word):
                if letter == guessed_letter:
                    self.masked_word[i] = letter
                    reward = self.correct_letter_reward
                    letter_found = True
            if not letter_found:
                self.lives_left -= 1
                reward = self.incorrect_letter_reward
            if "_" not in self.masked_word:
                done = True
                reward = self.win_reward
            elif self.lives_left <= 0:
                done = True
                reward = self.lose_reward
        return self._get_state(), reward, done, {}

    def _get_state(self):
        return {
            "masked_word": "".join(self.masked_word),
            "guessed_letters": sorted(list(self.guessed_letters)),
            "lives_left": self.lives_left,
            "hmm_probs": self.current_hmm_probs
        }

    def is_done(self):
        return "_" not in self.masked_word or self.lives_left <= 0

    def render(self):
        print(f"Word: {''.join(self.masked_word)}")
        print(f"Guessed Letters: {sorted(list(self.guessed_letters))}")
        print(f"Lives Left: {self.lives_left}")


def train_and_evaluate(hyperparameters, corpus_by_length, hmm_probabilities, test_data, max_word_length, seed=42):
    buffer_size = hyperparameters['buffer_size']
    batch_size = hyperparameters['batch_size']
    gamma = hyperparameters['gamma']
    lr = hyperparameters['lr']
    update_every = hyperparameters['update_every']
    epsilon_start = hyperparameters['epsilon_start']
    epsilon_decay = hyperparameters['epsilon_decay']
    epsilon_min = hyperparameters['epsilon_min']
    fc1_units = hyperparameters['fc1_units']
    fc2_units = hyperparameters['fc2_units']
    fc3_units = hyperparameters.get('fc3_units', None)
    n_episodes = hyperparameters['n_episodes']
    target_update_freq = hyperparameters['target_update_freq']
    hmm_weight = hyperparameters.get('hmm_weight', 0.0)
    rl_weight = hyperparameters.get('rl_weight', 1.0)
    correct_letter_reward = hyperparameters.get('correct_letter_reward', 1.0)
    incorrect_letter_reward = hyperparameters.get('incorrect_letter_reward', -1.0)
    win_reward = hyperparameters.get('win_reward', 5.0)
    lose_reward = hyperparameters.get('lose_reward', -5.0)
    repeated_letter_penalty = hyperparameters.get('repeated_letter_penalty', -0.1)

    masked_word_representation_size = max_word_length * (len(string.ascii_lowercase) + 1)
    guessed_letters_representation_size = len(string.ascii_lowercase)
    lives_left_representation_size = 1
    hmm_probs_representation_size = max_word_length * len(string.ascii_lowercase)
    state_size = (
        masked_word_representation_size +
        guessed_letters_representation_size +
        lives_left_representation_size +
        hmm_probs_representation_size
    )
    action_size = len(string.ascii_lowercase)

    env = HangmanEnv(corpus_by_length, hmm_probabilities, max_lives=6,
                     correct_letter_reward=correct_letter_reward,
                     incorrect_letter_reward=incorrect_letter_reward,
                     win_reward=win_reward,
                     lose_reward=lose_reward,
                     repeated_letter_penalty=repeated_letter_penalty)

    agent = DQNAgent(state_size=state_size, action_size=action_size, seed=seed,
                     buffer_size=buffer_size, batch_size=batch_size, gamma=gamma, lr=lr,
                     update_every=update_every, epsilon_start=epsilon_start, epsilon_decay=epsilon_decay, epsilon_min=epsilon_min,
                     fc1_units=fc1_units, fc2_units=fc2_units, fc3_units=fc3_units,
                     hmm_weight=hmm_weight, rl_weight=rl_weight)

    scores = []
    for i_episode in range(1, n_episodes + 1):
        state = env.reset()
        score = 0
        done = False
        numerical_state = state_to_numerical(state, max_len=max_word_length)
        while not done:
            action_index = agent.choose_action(numerical_state, state["guessed_letters"])
            action_letter = string.ascii_lowercase[action_index]
            next_state, reward, done, _ = env.step(action_letter)
            score += reward
            numerical_next_state = state_to_numerical(next_state, max_len=max_word_length)
            agent.step(numerical_state, action_index, reward, numerical_next_state, done)
            state = next_state
            numerical_state = numerical_next_state
            if i_episode % target_update_freq == 0:
                agent.update_target_network()
        scores.append(score)
        agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
        if i_episode % 1000 == 0:
            print(f'Episode {i_episode}/{n_episodes}, Average Score: {np.mean(scores[-1000:]):.2f}, Epsilon: {agent.epsilon:.4f}')

    total_score = 0
    correctly_guessed_words = 0
    print("\nStarting evaluation on the test set...")
    for word_to_guess in test_data:
        state = env.reset_for_eval(word_to_guess)
        numerical_state = state_to_numerical(state, max_len=max_word_length)
        done = False
        incorrect_guesses = 0
        game_won = False
        while not done:
            original_epsilon = agent.epsilon
            agent.epsilon = 0
            action_index = agent.choose_action(numerical_state, state["guessed_letters"])
            agent.epsilon = original_epsilon
            action_letter = string.ascii_lowercase[action_index]
            next_state, reward, done, _ = env.step(action_letter)
            if action_letter not in word_to_guess and action_letter not in state["guessed_letters"]:
                incorrect_guesses += 1
            if "_" not in next_state["masked_word"]:
                game_won = True
            numerical_next_state = state_to_numerical(next_state, max_len=max_word_length)
            state = next_state
            numerical_state = numerical_next_state
        if game_won:
            score = 10 * (6 - incorrect_guesses)
            total_score += score
            correctly_guessed_words += 1
        else:
            score = -10
            total_score += score

    evaluation_results = {
        "total_words": len(test_data),
        "correctly_guessed": correctly_guessed_words,
        "success_rate": correctly_guessed_words / len(test_data) * 100 if len(test_data) > 0 else 0,
        "total_score": total_score,
        "average_score": total_score / len(test_data) if len(test_data) > 0 else 0
    }

    print("\nEvaluation finished.")
    print(f"Correctly guessed words: {evaluation_results['correctly_guessed']}")
    print(f"Success Rate: {evaluation_results['success_rate']:.2f}%")
    print(f"Total score: {evaluation_results['total_score']}")
    print(f"Average score per word: {evaluation_results['average_score']:.2f}")

    return evaluation_results


## Run hyperparameter tuning

### Subtask:
Execute the implemented tuning strategy to train and evaluate the agent with various hyperparameter combinations.

**Reasoning**:
Iterate through the defined hyperparameter combinations and train and evaluate the agent for each set, storing the results for later analysis.


In [ ]:
evaluation_results_list = []

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"\n--- Training and evaluating with Hyperparameter Set {i+1} ---")
    print("Hyperparameters:", hp_set)
    results = train_and_evaluate(hp_set, corpus_by_length, hmm_probabilities, test_data, max_word_length)
    evaluation_results_list.append({'hyperparameters': hp_set, 'results': results})

print("\n--- Summary of Hyperparameter Tuning Results ---")
for result_entry in evaluation_results_list:
    print("\nHyperparameters:", result_entry['hyperparameters'])
    print("Evaluation Results:", result_entry['results'])

In [ ]:
evaluation_results_list = []

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"\n--- Training and evaluating with Hyperparameter Set {i+1} ---")
    print("Hyperparameters:", hp_set)
    results = train_and_evaluate(hp_set, corpus_by_length, hmm_probabilities, test_data, max_word_length)
    evaluation_results_list.append({'hyperparameters': hp_set, 'results': results})

print("\n--- Summary of Hyperparameter Tuning Results ---")
for result_entry in evaluation_results_list:
    print("\nHyperparameters:", result_entry['hyperparameters'])
    print("Evaluation Results:", result_entry['results'])

## Retrain the agent with new hyperparameters

### Subtask:
Retrain the agent with new hyperparameters by running the `train_and_evaluate` function with different combinations of hyperparameter values.

**Reasoning**:
Define a list of hyperparameter dictionaries and iterate through them, calling the `train_and_evaluate` function for each set and storing the results.


In [ ]:
hyperparameter_combinations = [
    {
        'buffer_size': int(1e5),
        'batch_size': 64,
        'gamma': 0.99,
        'lr': 5e-4,
        'update_every': 4,
        'epsilon_decay': 0.995,
        'epsilon_min': 0.01,
        'fc1_units': 128,
        'fc2_units': 128,
        'n_episodes': 10000,
        'target_update_freq': 100
    }
]

evaluation_results_list = []

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"\n--- Training and evaluating with Hyperparameter Set {i+1} ---")
    print("Hyperparameters:", hp_set)
    results = train_and_evaluate(hp_set, corpus_by_length, hmm_probabilities, test_data, max_word_length)
    evaluation_results_list.append({'hyperparameters': hp_set, 'results': results})

print("\n--- Summary of Hyperparameter Tuning Results ---")
for result_entry in evaluation_results_list:
    print("\nHyperparameters:", result_entry['hyperparameters'])
    print("Evaluation Results:", result_entry['results'])


--- Training and evaluating with Hyperparameter Set 1 ---
Hyperparameters: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 10000, 'target_update_freq': 100}
Episode 1000/10000, Average Score: -0.01, Epsilon: 0.01
Episode 2000/10000, Average Score: 0.94, Epsilon: 0.01
Episode 3000/10000, Average Score: 0.86, Epsilon: 0.01
Episode 4000/10000, Average Score: 1.17, Epsilon: 0.01
Episode 5000/10000, Average Score: 1.28, Epsilon: 0.01
Episode 6000/10000, Average Score: 1.48, Epsilon: 0.01
Episode 7000/10000, Average Score: 1.84, Epsilon: 0.01
Episode 8000/10000, Average Score: 1.97, Epsilon: 0.01
Episode 9000/10000, Average Score: 2.13, Epsilon: 0.01
Episode 10000/10000, Average Score: 2.19, Epsilon: 0.01

Starting evaluation on the test set...

Evaluation finished.
Correctly guessed words: 287
Success Rate: 14.35%
Total score: -10520
Average score per wor

# 🎯 Hyperparameter Analysis and Final Evaluation

---

## 🔍 Step 1: Analyze Results and Select Best Hyperparameters

### 🧩 Subtask:
Analyze the evaluation results obtained from various hyperparameter combinations and identify the **best performing configuration** based on metrics such as **success rate**, **average score**, and **overall stability**.

### 🧠 Objective:
Determine which hyperparameter set yields the **most consistent and highest-performing Hangman agent**.

---

## 🏁 Step 2: Final Evaluation with Best Hyperparameters

### 🧩 Subtask:
Conduct a **final training and evaluation** using the best hyperparameters identified from the tuning phase.  
Validate that the agent performs optimally and the results are **reproducible and stable**.

### 💡 Reasoning:
Re-run the training process using the top-performing hyperparameter configuration to:
- Confirm the improvement in success rate and average score.  
- Ensure that the model generalizes well across unseen test data.  
- Validate the robustness of the chosen hyperparameters.

---

✅ **Outcome:**
A refined DQN Hangman agent trained with optimized hyperparameters, demonstrating superior performance and reliability across evaluation metrics.


In [ ]:
best_hyperparameters = {
    'buffer_size': int(1e5),
    'batch_size': 64,
    'gamma': 0.99,
    'lr': 5e-4,
    'update_every': 4,
    'epsilon_decay': 0.995,
    'epsilon_min': 0.01,
    'fc1_units': 128,
    'fc2_units': 128,
    'n_episodes': 5000,
    'target_update_freq': 100
}

print("--- Running Final Evaluation with Best Hyperparameters ---")
print("Best Hyperparameters:", best_hyperparameters)

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

final_evaluation_results = train_and_evaluate(
    best_hyperparameters,
    corpus_by_length,
    hmm_probabilities,
    test_data,
    max_word_length
)

print("\n--- Final Evaluation Results (Best Hyperparameters) ---")
print(f"Total words in test set: {final_evaluation_results['total_words']}")
print(f"Correctly guessed words: {final_evaluation_results['correctly_guessed']}")
print(f"Success Rate: {final_evaluation_results['success_rate']:.2f}%")
print(f"Total score: {final_evaluation_results['total_score']}")
print(f"Average score per word: {final_evaluation_results['average_score']:.2f}")


--- Running Final Evaluation with Best Hyperparameters ---
Best Hyperparameters: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 5000, 'target_update_freq': 100}
Episode 1000/5000, Average Score: -0.01, Epsilon: 0.01
Episode 2000/5000, Average Score: 0.94, Epsilon: 0.01
Episode 3000/5000, Average Score: 0.86, Epsilon: 0.01
Episode 4000/5000, Average Score: 1.17, Epsilon: 0.01
Episode 5000/5000, Average Score: 1.28, Epsilon: 0.01

Starting evaluation on the test set...

Evaluation finished.
Correctly guessed words: 275
Success Rate: 13.75%
Total score: -11040
Average score per word: -5.52

--- Final Evaluation Results (Best Hyperparameters) ---
Total words in test set: 2000
Correctly guessed words: 275
Success Rate: 13.75%
Total score: -11040
Average score per word: -5.52


## 🏁 Final Evaluation with Best Hyperparameters

### 🎯 Subtask:
Run a final evaluation with the agent trained on the **best-performing hyperparameters** identified during tuning to validate the final model’s performance.

### 🧠 Reasoning:
After analyzing all tuning results, the agent should be retrained and tested using the **optimal hyperparameter configuration**.  
This step ensures that the final agent achieves **maximum success rate and stable performance** across the test dataset.


In [ ]:
best_hyperparameters = {
    'buffer_size': int(1e5),
    'batch_size': 64,
    'gamma': 0.99,
    'lr': 5e-4,
    'update_every': 4,
    'epsilon_decay': 0.995,
    'epsilon_min': 0.01,
    'fc1_units': 128,
    'fc2_units': 128,
    'n_episodes': 15000,
    'target_update_freq': 100
}

print("--- Running Final Evaluation with Best Hyperparameters ---")
print("Best Hyperparameters:", best_hyperparameters)

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

final_evaluation_results = train_and_evaluate(
    best_hyperparameters,
    corpus_by_length,
    hmm_probabilities,
    test_data,
    max_word_length
)

print("\n--- Final Evaluation Results (Best Hyperparameters) ---")
print(f"Total words in test set: {final_evaluation_results['total_words']}")
print(f"Correctly guessed words: {final_evaluation_results['correctly_guessed']}")
print(f"Success Rate: {final_evaluation_results['success_rate']:.2f}%")
print(f"Total score: {final_evaluation_results['total_score']}")
print(f"Average score per word: {final_evaluation_results['average_score']:.2f}")


--- Running Final Evaluation with Best Hyperparameters ---
Best Hyperparameters: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 15000, 'target_update_freq': 100}
Episode 1000/15000, Average Score: -0.01, Epsilon: 0.01
Episode 2000/15000, Average Score: 0.94, Epsilon: 0.01
Episode 3000/15000, Average Score: 0.86, Epsilon: 0.01
Episode 4000/15000, Average Score: 1.17, Epsilon: 0.01
Episode 5000/15000, Average Score: 1.28, Epsilon: 0.01
Episode 6000/15000, Average Score: 1.48, Epsilon: 0.01
Episode 7000/15000, Average Score: 1.84, Epsilon: 0.01
Episode 8000/15000, Average Score: 1.97, Epsilon: 0.01
Episode 9000/15000, Average Score: 2.13, Epsilon: 0.01
Episode 10000/15000, Average Score: 2.19, Epsilon: 0.01
Episode 11000/15000, Average Score: 2.15, Epsilon: 0.01
Episode 12000/15000, Average Score: 2.46, Epsilon: 0.01
Episode 13000/15000, Average Score: 

## 🧩 Analyze Results and Select Best Hyperparameters

### 🔹 Subtask:
Select the best hyperparameter configuration based on success rate and average score.

### 💡 Reasoning:
Identify the set of hyperparameters that achieved the highest performance metrics from the evaluation results.


In [ ]:
best_success_rate = -1
best_avg_score = -float('inf')
best_hyperparameters = None
best_results = None

print("Analyzing results to find the best hyperparameters...")
for result_entry in evaluation_results_list:
    hyperparameters = result_entry['hyperparameters']
    results = result_entry['results']
    success_rate = results['success_rate']
    average_score = results['average_score']

    print(f"\nChecking Hyperparameters: {hyperparameters}")
    print(f"  Success Rate: {success_rate:.2f}%")
    print(f"  Average Score: {average_score:.2f}")

    if success_rate > best_success_rate:
        best_success_rate = success_rate
        best_avg_score = average_score
        best_hyperparameters = hyperparameters
        best_results = results
    elif success_rate == best_success_rate and average_score > best_avg_score:
        best_avg_score = average_score
        best_hyperparameters = hyperparameters
        best_results = results

print("\n--- Best Hyperparameters Found ---")
if best_hyperparameters:
    print("Hyperparameters:", best_hyperparameters)
    print("Evaluation Results:", best_results)
else:
    print("No evaluation results found.")


Analyzing results to find the best hyperparameters...

Checking Hyperparameters: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 5000, 'target_update_freq': 100}
  Success Rate: 13.75%
  Average Score: -5.52

Checking Hyperparameters: {'buffer_size': 50000, 'batch_size': 32, 'gamma': 0.95, 'lr': 0.001, 'update_every': 10, 'epsilon_decay': 0.99, 'epsilon_min': 0.05, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 5000, 'target_update_freq': 50}
  Success Rate: 9.60%
  Average Score: -6.63

Checking Hyperparameters: {'buffer_size': 200000, 'batch_size': 128, 'gamma': 0.999, 'lr': 0.0001, 'update_every': 1, 'epsilon_decay': 0.998, 'epsilon_min': 0.005, 'fc1_units': 256, 'fc2_units': 256, 'n_episodes': 5000, 'target_update_freq': 200}
  Success Rate: 10.45%
  Average Score: -6.83

--- Best Hyperparameters Found ---
Hyperparameters: {'buffer_size': 100000,

## 🔧 Define Hyperparameter Search Space

### 🧩 Subtask:
Specify the range of potential values for each key hyperparameter that influences the DQN agent’s performance.

**Reasoning:**
Establish a collection of hyperparameter dictionaries to explore during tuning.  
These combinations will represent different configurations of learning rate, discount factor, batch size, and other parameters — enabling effective experimentation through random or manual search.


In [ ]:
hyperparameter_combinations = [
    {
        'buffer_size': int(1e5),
        'batch_size': 64,
        'gamma': 0.99,
        'lr': 5e-4,
        'update_every': 4,
        'epsilon_decay': 0.995,
        'epsilon_min': 0.01,
        'fc1_units': 64,
        'fc2_units': 64,
        'n_episodes': 10000,
        'target_update_freq': 100
    },
    {
        'buffer_size': int(5e4),
        'batch_size': 32,
        'gamma': 0.95,
        'lr': 1e-3,
        'update_every': 10,
        'epsilon_decay': 0.99,
        'epsilon_min': 0.05,
        'fc1_units': 64,
        'fc2_units': 64,
        'n_episodes': 5000,
        'target_update_freq': 50
    },
    {
        'buffer_size': int(2e5),
        'batch_size': 128,
        'gamma': 0.999,
        'lr': 1e-4,
        'update_every': 1,
        'epsilon_decay': 0.998,
        'epsilon_min': 0.005,
        'fc1_units': 128,
        'fc2_units': 128,
        'n_episodes': 5000,
        'target_update_freq': 200
    }
]

print("Defined hyperparameter combinations for tuning:")
for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"Set {i+1}: {hp_set}")


Defined hyperparameter combinations for tuning:
Set 1: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 10000, 'target_update_freq': 100}
Set 2: {'buffer_size': 50000, 'batch_size': 32, 'gamma': 0.95, 'lr': 0.001, 'update_every': 10, 'epsilon_decay': 0.99, 'epsilon_min': 0.05, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 5000, 'target_update_freq': 50}
Set 3: {'buffer_size': 200000, 'batch_size': 128, 'gamma': 0.999, 'lr': 0.0001, 'update_every': 1, 'epsilon_decay': 0.998, 'epsilon_min': 0.005, 'fc1_units': 128, 'fc2_units': 128, 'n_episodes': 5000, 'target_update_freq': 200}


## 🔍 Run Hyperparameter Tuning  

### 🎯 Subtask:  
Train and evaluate the agent across multiple hyperparameter combinations to identify the best-performing setup.  

**🧠 Reasoning:**  
Loop through the defined hyperparameter sets, train the agent for each configuration, and record the per


In [ ]:
evaluation_results_list = []

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"\n--- Training and evaluating with Hyperparameter Set {i+1} ---")
    print("Hyperparameters:", hp_set)
    results = train_and_evaluate(hp_set, corpus_by_length, hmm_probabilities, test_data, max_word_length)
    evaluation_results_list.append({'hyperparameters': hp_set, 'results': results})

print("\n--- Summary of Hyperparameter Tuning Results ---")
for result_entry in evaluation_results_list:
    print("\nHyperparameters:", result_entry['hyperparameters'])
    print("Evaluation Results:", result_entry['results'])



--- Training and evaluating with Hyperparameter Set 1 ---
Hyperparameters: {'buffer_size': 100000, 'batch_size': 64, 'gamma': 0.99, 'lr': 0.0005, 'update_every': 4, 'epsilon_decay': 0.995, 'epsilon_min': 0.01, 'fc1_units': 64, 'fc2_units': 64, 'n_episodes': 10000, 'target_update_freq': 100}
Episode 1000/10000, Average Score: -0.56, Epsilon: 0.01
Episode 2000/10000, Average Score: 1.29, Epsilon: 0.01
Episode 3000/10000, Average Score: 1.98, Epsilon: 0.01
Episode 4000/10000, Average Score: 1.98, Epsilon: 0.01
Episode 5000/10000, Average Score: 1.36, Epsilon: 0.01
Episode 6000/10000, Average Score: 2.01, Epsilon: 0.01
Episode 7000/10000, Average Score: 1.42, Epsilon: 0.01
Episode 8000/10000, Average Score: 2.44, Epsilon: 0.01
Episode 9000/10000, Average Score: 1.71, Epsilon: 0.01
Episode 10000/10000, Average Score: 1.61, Epsilon: 0.01

Starting evaluation on the test set...

Evaluation finished.
Correctly guessed words: 315
Success Rate: 15.75%
Total score: -9990
Average score per word: 

In [ ]:
evaluation_results_list = []

max_word_length = max(corpus_by_length.keys()) if corpus_by_length else 24

hyperparameter_combinations = [
    {
        'n_episodes': 50000,
        'target_update_freq': 500,
        'fc1_units': 256,
        'fc2_units': 256,
        'fc3_units': 128,
        'buffer_size': int(5e5),
        'batch_size': 128,
        'gamma': 0.99,
        'lr': 1e-4,
        'update_every': 4,
        'epsilon_start': 1.0,
        'epsilon_decay': 0.9995,
        'epsilon_min': 0.05,
        'hmm_weight': 0.4,
        'rl_weight': 0.6,
        'correct_letter_reward': 2.0,
        'incorrect_letter_reward': -1.5,
        'win_reward': 20.0,
        'lose_reward': -15.0,
        'repeated_letter_penalty': -2.0,
    }
]

for i, hp_set in enumerate(hyperparameter_combinations):
    print(f"\n--- Training and evaluating with Hyperparameter Set {i+1} ---")
    print("Hyperparameters:", hp_set)
    results = train_and_evaluate(hp_set, corpus_by_length, hmm_probabilities, test_data, max_word_length)
    evaluation_results_list.append({'hyperparameters': hp_set, 'results': results})

print("\n--- Summary of Hyperparameter Tuning Results ---")
for result_entry in evaluation_results_list:
    print("\nHyperparameters:", result_entry['hyperparameters'])
    print("Evaluation Results:", result_entry['results'])



--- Training and evaluating with Hyperparameter Set 1 ---
Hyperparameters: {'n_episodes': 50000, 'target_update_freq': 500, 'fc1_units': 256, 'fc2_units': 256, 'fc3_units': 128, 'buffer_size': 500000, 'batch_size': 128, 'gamma': 0.99, 'lr': 0.0001, 'update_every': 4, 'epsilon_start': 1.0, 'epsilon_decay': 0.9995, 'epsilon_min': 0.05, 'hmm_weight': 0.4, 'rl_weight': 0.6, 'correct_letter_reward': 2.0, 'incorrect_letter_reward': -1.5, 'win_reward': 20.0, 'lose_reward': -15.0, 'repeated_letter_penalty': -2.0}
Episode 1000/50000, Average Score: -13.69, Epsilon: 0.6065
Episode 2000/50000, Average Score: -8.41, Epsilon: 0.3678
Episode 3000/50000, Average Score: -4.12, Epsilon: 0.2230
Episode 4000/50000, Average Score: -1.41, Epsilon: 0.1353
Episode 5000/50000, Average Score: 2.22, Epsilon: 0.0820
Episode 6000/50000, Average Score: 2.69, Epsilon: 0.0500
Episode 7000/50000, Average Score: 4.20, Epsilon: 0.0500
Episode 8000/50000, Average Score: 3.67, Epsilon: 0.0500
Episode 9000/50000, Average